## Ensemble learning
- diversity is conducive to improving ensemble learning scores
- This ensemble weight combination is verified on the validation set. 
    - If there are enough test opportunities, it should be able to improve

In [4]:
import pickle
import numpy as np
from sklearn.metrics import f1_score
import torch

import numpy as np

def top_k_accuracy(scores, true_labels):
    top_5_predictions = np.argsort(scores, axis=1)[:, -5:]
    top_1_predictions = top_5_predictions[:, -1:]

    correct_top_1 = 0
    correct_top_5 = 0
    total = len(true_labels)

    for i in range(total):
        correct_top_1 += top_1_predictions[i] == true_labels[i]
        correct_top_5 += np.any(top_5_predictions[i] == true_labels[i])

    micro_accuracy_top_1 = correct_top_1 / total
    micro_accuracy_top_5 = correct_top_5 / total

    return micro_accuracy_top_1[0], micro_accuracy_top_5

def fine2coarse(x):
    if x <= 4:
        return 0
    elif 5 <= x <= 10:
        return 1
    elif 11 <= x <= 23:
        return 2
    elif 24 <= x <= 31:
        return 3
    elif 32 <= x <= 37:
        return 4
    elif 38 <= x <= 47:
        return 5
    else:
        return 6

def lv_evaluation(predictions, labels):
    # predictions = np.argsort(predictions, axis=1)[:, -1].tolist()
    predictions = predictions.tolist()
    coarse_preds = [fine2coarse(lv2id) for lv2id in predictions]
    coarse_labels = [fine2coarse(lv2id) for lv2id in labels]
    fine_f1_micro = f1_score(labels, predictions, average='micro')
    fine_f1_macro = f1_score(labels, predictions, average='macro')
    coarse_f1_micro = f1_score(coarse_labels, coarse_preds, average='micro')
    coarse_f1_macro = f1_score(coarse_labels, coarse_preds, average='macro')
    f1_mean = (fine_f1_macro + coarse_f1_macro + coarse_f1_micro + fine_f1_micro) / 4.0
    return f1_mean


paths = [
        './output/ModelGroup-xl/swin_test0708.pickle', # 0.708
         './output/ModelGroup3/swin_base_ssv2_crop_alltrick_addval_epoch36_untest.pickle', # 0.715
         './output/ModelGroup4/swin_Large_K700_crop_alltrick_addval_epoch33_untest.pickle', # 0.710
         
         './output/ModelGroup1/mae_crop_alltrick_addval_epoch42_untest.pickle', # 0.728
         './output/ModelGroup2/TSN/mae_crop_alltrick_coarsehead_tsn_addval_epoch42_untest.pickle', # 0.731
         './output/ModelGroup-xl/mae_test0711.pickle', # 0.711
         './output/ModelGroup5/mae_of_val07218_addval_epoch42_untest.pickle', # 0.715
         './output/ModelGroup-xl/mae_base_ssv2_train_crop_val_6876_fortest.pickle', # 0.689
         './output/ModelGroup-xl/mae_base_k710_trainval_crop_test_719_fortest.pickle', # 0.719
         
         './output/ModelGroup-xl/mae_base_rgb_flow.pickle', # 0.721
         
         './output/ModelGroup-xl/mae_large_k700_trainval_crop_fortest.pickle', # 0.730
         
         './output/ModelGroup-xl/mae_huge_k700_trainval_crop_val_7169_fortest.pickle', #  0.726
         
         './output/ModelGroup-xl/umt_trainval_crop_val_7067_fortest.pickle', # 0.723
    
         './output/ModelGroup-xl/intervideo_baseline_val_7102_fortest699.pickle' # 0.698
        ]
weights =  [0.7,0.9,0.7,0.4,0.4,0.2,0.3,0.6,0.4,0.9,0.6,0.7,0.8,0.7]
    

pred_scores = []
labels = []

for i in range(len(paths)):
    with open(paths[i], 'rb') as file:
        datas = pickle.load(file)
    pred_scores.append([])
    for data in datas:
        pred_scores[i].append(data['pred_score'].numpy())
        if i == 0:
            labels.append(data['gt_label'].numpy())
pred_scores = np.array(pred_scores)
labels = np.array(labels)

assemble_scores = 0
for i in range(len(paths)):
    assemble_scores += pred_scores[i] * weights[i]
predicted_labels = np.argmax(assemble_scores, axis=1)
top1_acc, top5_acc = top_k_accuracy(assemble_scores, labels)
mean_f1 = lv_evaluation(predicted_labels, labels)
print(f"Top-1 Accuracy: {top1_acc:.6f}, Top-5 Accuracy: {top5_acc:.6f}, Mean F1 Score: {mean_f1:.6f}")

with open('./output/prediction_ensemble.pickle', 'wb') as file:
    results = []
    for pl in predicted_labels:
        results.append({
            'pred_label': torch.tensor([pl]),
        })
    pickle.dump(results, file)

Top-1 Accuracy: 0.030756, Top-5 Accuracy: 0.156415, Mean F1 Score: 0.024752


## Prepare submission files

In [5]:
'''

Packaging to generate zip files for uploading, 
the default coarse-grained labels are derived from the fine-grained labels, 
if you use separately generated coarse-grained labels, be sure to modify the relevant code.
'''
import os
import pickle
import zipfile
import csv

pickle_file_path = './output/prediction_ensemble.pickle'
csv_file_path = './output/prediction.csv'
zip_file_path = './output/ensemble.zip'


def fine2coarse(x):
    if x <= 4:
        return 0
    elif 5 <= x <= 10:
        return 1
    elif 11 <= x <= 23:
        return 2
    elif 24 <= x <= 31:
        return 3
    elif 32 <= x <= 37:
        return 4
    elif 38 <= x <= 47:
        return 5
    else:
        return 6


with open(pickle_file_path, 'rb') as file:
    datas = pickle.load(file)
f = open(csv_file_path, 'w', newline='')
writer = csv.writer(f)
writer.writerow(["id", "pred_label_1", "pred_label_2"])
for index, data in enumerate(datas):
    file_name = "test" + str(index).zfill(4) + '.mp4'
    fine_pred = data['pred_label'].cpu().numpy()[0]
    coarse_pred = fine2coarse(fine_pred)
    writer.writerow([file_name, str(coarse_pred), str(fine_pred)])
f.close()
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    zipf.write(csv_file_path, os.path.basename(csv_file_path))
